In [1]:
import asyncio
import importlib
import json
import time
from copy import deepcopy
from datetime import datetime
from pprint import pprint
from time import perf_counter
from typing import *
from uuid import UUID

from aiohttp import ClientResponse

from vindicator.database.vindicator_database import VindicatorDatabase
from vindicator.request.fetch_guild import FetchedGuild
from vindicator.request.fetch_player import FetchedPlayer
from vindicator.utils.wynncraft_response_utils import WynncraftResponseUtils as WynnUtils

async def astart():
    await WynncraftAPIRequest.ainit()
    await VindicatorDatabase.ainit()

async def aclose():
    await VindicatorDatabase._close_connection()
    await WynncraftAPIRequest._session._close()

asyncio.create_task(astart())

fetched_guilds: List[FetchedGuild]
fetched_players: List[FetchedPlayer]
with open("fetched_guild.json") as f:
    fetched_guilds = json.load(f)
with open("fetched_player.json") as f:
    fetched_players = json.load(f)

In [ ]:
def test1():
    query1 = "INSERT INTO test (timestamp) VALUES (%(timestamp)s)"
    params1 = [{"timestamp": datetime.fromtimestamp(ts)} for ts in range(int(time.time()), int(time.time()) + 1_000_000)]
    coro1 = VindicatorDatabase.write_many(query1, params1)
    task1 = asyncio.create_task(coro1)
    return task1

def test2():
    query2 = "INSERT INTO test_int (timestamp) VALUES (%(timestamp)s)"
    params2 = [{"timestamp": ts} for ts in range(int(time.time()), int(time.time()) + 1_000_000)]
    coro2 = VindicatorDatabase.write_many(query2, params2)
    task2 = asyncio.create_task(coro2)
    return task2

def test3():
    query = "SELECT timestamp FROM test LIMIT 10"
    coro1 = VindicatorDatabase.read_all(query)
    task1 = asyncio.create_task(coro1)
    return task1

def test4():
    query = "SELECT timestamp FROM test_int LIMIT 10"
    coro1 = VindicatorDatabase.read_all(query)
    task1 = asyncio.create_task(coro1)
    return task1

def test5():
    query = (
        "INSERT INTO player_main_info (uuid, latest_username, timestamp) "
        "VALUES (%(uuid)s, %(latest_username)s, %(timestamp)s)"
    )
    with open("uuid-username-timestamp.json", "r") as f:
        data = json.load(f)

    for record in data:
        record["uuid"] = UUID(record["uuid"]).bytes

    params = data
    coro = VindicatorDatabase.write_many(query, params)
    task = asyncio.create_task(coro)
    return task

def test6(name: str):
    coro = WynncraftAPIRequest.get_player_stats_json(name)
    task = asyncio.create_task(coro)
    return task

async def test7(username: str):
    params = {'username': username}
    query = f"SELECT uuid FROM {VindicatorTables.PLAYER_MAIN_INFO} WHERE latest_username = %(username)s"
    uuid = await VindicatorDatabase.read_all(query=query, params=params)
    return uuid

In [2]:
dict_data = {
    "guild_main_info": GuildMainInfo.from_raw(fetched_guilds),
    "guild_main": GuildMain.from_raw(fetched_guilds),
    "guild_member": GuildMember.from_raw(fetched_guilds),
    "player_character_info": PlayerCharacterInfo.from_raw(fetched_players),
    "player_character": PlayerCharacter.from_raw(fetched_players),  # BUG
    "player_main_info": PlayerMainInfo.from_raw(fetched_players),
    "player_main": PlayerMain.from_raw(fetched_players),
    # "player_uptime": PlayerUptime.from_raw(fetched_player),
    # "raw_latest_others": RawLatestOthers.from_raw(fetched_player),
    "raw_online_guild": RawOnlineGuild.from_raw(fetched_guilds),
    "raw_online_player": RawOnlinePlayer.from_raw(fetched_players),
    "raw_recent_guild": RawRecentGuild.from_raw(fetched_guilds),
    "raw_recent_player": RawRecentPlayer.from_raw(fetched_players),
}

In [3]:
async def insert():
    await GuildMainInfo.to_db(dict_data["guild_main_info"])
    await PlayerMainInfo.to_db(dict_data["player_main_info"])
    await PlayerCharacterInfo.to_db(dict_data["player_character_info"])
    await GuildMain.to_db(dict_data["guild_main"])
    await GuildMember.to_db(dict_data["guild_member"])
    await PlayerCharacter.to_db(dict_data["player_character"])
    await PlayerMain.to_db(dict_data["player_main"])
    # await PlayerUptime.to_db(dict_data["player_uptime"])
    # await RawLatestOthers.to_db(dict_data["raw_latest_others"])
    await RawOnlineGuild.to_db(dict_data["raw_online_guild"])
    await RawOnlinePlayer.to_db(dict_data["raw_online_player"])
    await RawRecentGuild.to_db(dict_data["raw_recent_guild"])
    await RawRecentPlayer.to_db(dict_data["raw_recent_player"])

In [12]:
task = asyncio.create_task(insert())

In [17]:
print(task.done())
print(task.result())

True
None


In [15]:
for k, v in dict_data.items():
    print(k, list(v[0].keys()), '\n')

guild_main_info
['created', 'name', 'prefix']

guild_main
['level', 'member_total', 'name', 'online_members', 'territories', 'wars', 'unique_hash']

guild_member
['joined', 'uuid', 'contributed']

player_character_info
['charcter_uuid', 'type', 'uuid']

player_character
['character_uuid', 'chests_found', 'deaths', 'discoveries', 'level', 'logins', 'mobs_killed', 'playtime', 'wars', 'xp', 'dungeon_completions', 'quest_completions', 'raid_completions', 'gamemode', 'fishing', 'woodcutting', 'mining', 'farming', 'scribing', 'jeweling', 'alchemism', 'cooking', 'weaponsmithing', 'tailoring', 'woodworking', 'armouring', 'unique_hash', 'timestamp']

player_main_info
['first_join', 'latest_username', 'server', 'timestamp', 'uuid']

player_main
['guild_name', 'guild_rank', 'playtime', 'support_rank', 'username', 'uuid', 'unique_hash', 'timestamp']

raw_online_guild
['guild_name', 'response']

raw_online_player
['response', 'uuid']

raw_recent_guild
['guild_name', 'response', 'timestamp']

raw_re

In [27]:
print(t.done())
print(t.result())

True


AssertionError: 